In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

Data Preparation

In [19]:
# Load ratings data
ratings_data = pd.read_csv("/Users/yushiyang/desktop/RecSys-Materials/ml-latest-small/ratings.csv")

# Identify unique users and movies
unique_users = ratings_data['userId'].unique()
unique_movies = ratings_data['movieId'].unique()

# Create a user-movie matrix
user_movie_matrix = np.zeros((len(unique_users), len(unique_movies)))

# Fill the user-movie matrix
for _, row in ratings_data.iterrows():
    user_idx = np.where(unique_users == row['userId'])[0][0]
    movie_idx = np.where(unique_movies == row['movieId'])[0][0]
    user_movie_matrix[user_idx, movie_idx] = 1

print(user_movie_matrix)
print("Size of user-movie matrix:", user_movie_matrix.shape)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Size of user-movie matrix: (671, 9066)


Implement Minhash from scratch

In [20]:
# Generates a list of random hash functions, each represented by a pair (a, b)
# Allow seed setting to recreate the hash functions/permutations
def generate_hash_functions(num_perm, seed=None):
    if seed is not None:
        np.random.seed(seed)  # Set the seed for the random number generator
    hash_functions = []
    for _ in range(num_perm):
        a = np.random.randint(1, 2**32 - 1)  # Generate a random 'a' value
        b = np.random.randint(0, 2**32 - 1)  # Generate a random 'b' value
        hash_functions.append((a, b))  # Append the hash function parameters to the list
    return hash_functions

# Computes MinHash values for a given vector using a list of hash functions
def Minhash(vector, hash_functions):
    minhash_values = []
    for a, b in hash_functions:  # Iterate through all hash functions (permutations)
        min_hash = float('inf')  # Set the initial min_hash to positive infinity
        for idx, value in enumerate(vector):
            if value == 1:  # If the value is 1 (indicating the presence of an item)
                hash_val = (a * idx + b) % len(vector)  # Compute the hash value (use a simple hash function for randomisation)
                min_hash = min(min_hash, hash_val)
        minhash_values.append(min_hash)  # Append the computed min_hash value to the list
    return minhash_values

Implement LSH from scratch

In [22]:
class LSH:
    def __init__(self, num_bands, band_size):
        self.num_bands = num_bands  # Number of bands for LSH
        self.band_size = band_size  # Number of rows in each band
        self.buckets = defaultdict(set)  # Dictionary to store buckets
    
    def hash_band(self, band):
        return hash(tuple(band))  # Generate a hash value for a band (simple hash function)
    
    def insert(self, user_id, minhash):
        for i in range(0, num_perm, self.band_size):  # Iterate through the permutations in bands
            band = minhash[i:i+self.band_size]  # Extract a band of MinHash values
            band_hash = self.hash_band(band)  # Hash the band
            self.buckets[band_hash].add(user_id)  # Add user to the corresponding bucket
    
    def query(self, minhash_query):
        similar_users = set()
        for i in range(0, num_perm, self.band_size):  # Iterate through the permutations in bands
            band = minhash_query[i:i+self.band_size]  # Extract a band from the query MinHash
            band_hash = self.hash_band(band)  # Hash the band
            similar_users.update(self.buckets[band_hash])  # Update similar users from the corresponding bucket
        return similar_users

In [23]:
# Define the length of the user vector
vector_length = 9066 # Number of movies

# Generate a random user vector with 0s and 1s
random_user_vector = np.random.randint(2, size=vector_length)
print(random_user_vector)

[1 0 0 ... 0 1 1]


Compute minhash vectors for all users - much slower than datasketch library implementation

In [25]:
num_perm = 128  # Number of permutation functions
hash_functions = generate_hash_functions(num_perm, seed=100)  # Generate a list of random hash functions (the same across all users)

# Compute MinHash values for each user
all_minhashes_for_users = []
for user_vector in user_movie_matrix:
    one_user_minhash = Minhash(user_vector, hash_functions)
    all_minhashes_for_users.append(one_user_minhash)

print(all_minhashes_for_users[0], len(all_minhashes_for_users))

[660, 35, 641, 82, 74, 424, 26, 196, 372, 573, 69, 161, 78, 135, 77, 1192, 927, 135, 309, 530, 197, 145, 319, 183, 88, 465, 188, 245, 463, 167, 441, 177, 348, 213, 61, 206, 372, 125, 426, 692, 2, 490, 176, 495, 103, 57, 210, 7, 528, 458, 196, 202, 266, 970, 445, 187, 50, 51, 823, 576, 1279, 334, 299, 191, 75, 466, 467, 216, 411, 574, 94, 65, 579, 2139, 407, 93, 294, 108, 861, 434, 70, 298, 250, 901, 29, 651, 225, 651, 1516, 30, 1176, 1026, 457, 430, 2536, 22, 199, 777, 315, 52, 70, 624, 249, 1279, 9, 306, 156, 490, 84, 343, 1966, 275, 8, 75, 1160, 39, 962, 327, 86, 549, 435, 1774, 248, 712, 361, 964, 225, 698] 671


In [24]:
# Compute MinHash values for the user vector
minhash_query = Minhash(random_user_vector, hash_functions)

# Create an LSH index
num_bands = 32  # Number of bands for LSH
band_size = num_perm // num_bands  # The smaller, more objects in the same bucket
lsh = LSH(num_bands, band_size)  # Create an LSH object

# Insert MinHashes into the LSH index
for idx, minhash in enumerate(all_minhashes_for_users):  # Loop through each MinHash signature
    lsh.insert(str(idx), minhash)  # Insert the MinHash signature into the LSH index

# Query LSH to find similar users
similar_users = lsh.query(minhash_query)  # Query for similar users based on a query MinHash signature
print(similar_users)

{'563', '29', '14', '623', '546'}
